In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import exoplanet as xo

import theano.tensor as tt
from exoplanet.theano_ops.starry.integrated_limbdark import IntegratedLimbDarkOp
op = IntegratedLimbDarkOp(tol=1e-12, min_depth=3, max_depth=50, include_contacts=False)

orbit = xo.orbits.KeplerianOrbit(period=1.0, b=0.5)
t = np.linspace(-1, 1, 10000)
texp = 0.01
u = [0.3, 0.2]
star = xo.StarryLightCurve(u)

x, y, z = orbit.get_relative_position(t)
dx, dy, dz = orbit.get_relative_velocity(t)
d2x, d2y, d2z = orbit.get_relative_acceleration(t)
b = tt.sqrt(x**2 + y**2)
db = (x * dx + y * dy) / b
d2b = (dx**2 + dy**2 - db**2 + x*d2x + y*d2y) / b

r = 0.1 + tt.zeros_like(b)
dt = texp + tt.zeros_like(b)

res = op(star.c_norm, r, x, dx, d2x, y, dy, d2y, z, dz, dt)

light_curve_exact = star.get_light_curve(
    orbit=orbit, r=0.1, t=t, texp=texp, oversample=1000).eval()
plt.plot(t, light_curve_exact)

light_curve = star.get_light_curve(order=0,
    orbit=orbit, r=0.1, t=t, texp=texp, oversample=15).eval()
plt.plot(t, light_curve)

plt.plot(t, res[0].eval())

In [ ]:
res[-1].eval(), len(t) * 15

In [ ]:
plt.plot(t, np.log10(np.abs(light_curve - light_curve_exact)))
plt.plot(t, np.log10(np.abs(res[0].eval() - light_curve_exact[:, 0])), label="quad");
# plt.plot(t, np.log10(np.abs(res0[0].eval() - light_curve_exact[:, 0])), label="linear");
plt.legend();

In [ ]:
plt.plot(t, orbit._get_true_anomaly(t).eval())

In [ ]:
plt.plot(t, x.eval())
plt.plot(t, dx.eval())
plt.plot(t, d2x.eval())

In [ ]:
plt.plot(t, y.eval())
plt.plot(t, dy.eval())
plt.plot(t, d2y.eval())

In [ ]:
be = db.eval()
plt.plot(t[1:-1], 0.5*(be[2:] - be[:-2]) / np.diff(t[:-1]))
plt.plot(t, d2b.eval())

In [ ]:
r = 0.1 + tt.zeros_like(b)
los = tt.ones_like(b)
dt = texp + tt.zeros_like(b)

res = op(star.c_norm, b, r, los, db, d2b, dt)

plt.plot(t, res[0].eval())
plt.plot(t, light_curve_exact)

light_curve_exact = star.get_light_curve(
    orbit=orbit, r=0.1, t=t, texp=texp, oversample=1000).eval()
light_curve = star.get_light_curve(order=2,
    orbit=orbit, r=0.1, t=t, texp=texp, oversample=7).eval()
plt.plot(t, light_curve)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import exoplanet as xo

# The light curve calculation requires an orbit
orbit = xo.orbits.KeplerianOrbit(period=1)

# Compute a limb-darkened light curve using starry
texp = 0.1
t = np.linspace(0.0, 0.1, 1000)
u = [0.3, 0.2]
star = xo.StarryLightCurve(u)
light_curve_instant = star.get_light_curve(
    orbit=orbit, r=0.1, t=t).eval()
light_curve_exact = star.get_light_curve(
    orbit=orbit, r=0.1, t=t, texp=texp, oversample=1000).eval()

fig, axes = plt.subplots(4, 1, figsize=(5, 10), sharex=True)

ax = axes[0]
ax.plot(t, light_curve_instant * 1e3, ":k")
ax.plot(t, light_curve_exact * 1e3, "k")
ax.set_ylabel("relative flux [ppt]")

for n in [3, 5, 7, 15, 51][::-1]:
    for order in range(3):
        ax = axes[order+1]
        light_curve = star.get_light_curve(order=order,
            orbit=orbit, r=0.1, t=t, texp=texp, oversample=n).eval()
        ax.plot(t, np.log10(np.abs(light_curve - light_curve_exact)),
                label="k = {0}".format(n))

for i, ax in enumerate(axes[1:]):
    ax.annotate("order = {0}".format(i), (0, 1), xycoords="axes fraction",
                ha="left", va="top",
                xytext=(5, -10), textcoords="offset points",
                fontsize=10)
    for loc, name in [(-3, "ppt"), (-6, "ppm"), (-9, "ppb")]:
        ax.axhline(loc, color="k", alpha=0.3, lw=1)
        ax.annotate(name, (t.max(), loc), va="top", ha="right",
                    xytext=(-3, -2), textcoords="offset points",
                    fontsize=10, alpha=0.3)
    ax.set_ylim(-10.2, -2.7)
    ax.set_ylabel("log$_{10}$(flux error)")

ax.set_xlabel("time [days]")
ax.set_xlim(t.min(), t.max())
fig.subplots_adjust(hspace=0.0);

In [ ]:
plt.plot(t, np.log10(np.abs(res[0].eval() - light_curve_exact[:, 0])))